# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [40]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
#Loaded CSV file from WeatherPy
cities_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
cities_data.head()

,Unnamed: 0,City Name,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hilo,19.7297,-155.0900,71.60,73,90,5.75,US,1619462641
1,1,saint-georges,12.0564,-61.7485,86.00,66,20,24.16,GD,1619462721
2,2,butaritari,3.0707,172.7902,83.19,74,85,13.76,KI,1619462722
3,3,mount isa,-20.7333,139.5000,75.20,46,56,9.22,AU,1619462722
4,4,tasiilaq,65.6145,-37.6368,39.20,75,75,9.93,GL,1619462722


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [5]:
#Stored the Lat and Long columns into locations and Humidity into humidity
locations = cities_data[["Lat", "Long"]].astype(float)
humidity = cities_data["Humidity"].astype(float)

In [12]:
#Created Heatlayer Map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius=2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
#Filtered the dataframe by ideal weather conditions
hotel_df = cities_data.loc[(cities_data["Max Temp"] > 70) &
                                       (cities_data["Max Temp"] < 80) &
                                       (cities_data["Wind Speed"] < 10) &
                                       (cities_data["Cloudiness"] == 0)]
hotel_df.dropna().head()

,Unnamed: 0,City Name,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
274,274,puerto madryn,-42.7692,-65.0385,70.65,35,0,4.52,AR,1619462803
288,288,bell ville,-32.6259,-62.6887,73.00,49,0,1.99,AR,1619462807
415,415,santa fe,-31.6333,-60.7000,75.00,42,0,4.00,AR,1619462845
499,499,inhambane,-23.8650,35.3833,72.59,84,0,5.14,MZ,1619462870
516,516,rafaela,-31.2503,-61.4867,72.00,32,0,1.01,AR,1619462875


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
#Added a new column to the dataframe called Hotel Name
hotel_df["Hotel Name"] =  ""

hotel_df.head(15)

<ipython-input-26-a11a23693181>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] =  ""


,Unnamed: 0,City Name,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
274,274,puerto madryn,-42.7692,-65.0385,70.65,35,0,4.52,AR,1619462803,
288,288,bell ville,-32.6259,-62.6887,73.00,49,0,1.99,AR,1619462807,
415,415,santa fe,-31.6333,-60.7000,75.00,42,0,4.00,AR,1619462845,
499,499,inhambane,-23.8650,35.3833,72.59,84,0,5.14,MZ,1619462870,
516,516,rafaela,-31.2503,-61.4867,72.00,32,0,1.01,AR,1619462875,
558,558,cockburn town,21.4612,-71.1419,78.60,72,0,9.01,TC,1619462888,


In [78]:
#Created a dictionary of parameters to pas in the get request
params ={
    "rankby": "distance",
    "type": "lodging",
    "key": g_key,
    "keyword" : "hotel",    
}

#Used a for loop with .iterrows to iterate through the dataframe
for index, row in hotel_df.iterrows():
    #Created a base URL from the Google Place documentation
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #retrieved the latitude and longitude from the dataframe
    lat = row["Lat"]
    lng = row["Long"]
    #Added location to the parameters dictionary and assinging lat ad lng to it
    params["location"] = f"{lat},{lng}"
    #Assembled the URL using the base URL and parameters dictionary 
    response = requests.get(base_url, params=params)
    response = response.json()
    #extracted the results and added them to the dataframe Hotel Name column
    hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
#Visualized the dataframe with the new data added
hotel_df

C:\Users\dkrug\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Unnamed: 0,City Name,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
274,274,puerto madryn,-42.7692,-65.0385,70.65,35,0,4.52,AR,1619462803,El Gualicho Hostel
288,288,bell ville,-32.6259,-62.6887,73.00,49,0,1.99,AR,1619462807,Hotel Los Inmigrantes
415,415,santa fe,-31.6333,-60.7000,75.00,42,0,4.00,AR,1619462845,Ámbit Boulevard Hotel Boutique
499,499,inhambane,-23.8650,35.3833,72.59,84,0,5.14,MZ,1619462870,Hotel Casa Do Capitão
516,516,rafaela,-31.2503,-61.4867,72.00,32,0,1.01,AR,1619462875,Toscano Hotel & Café
558,558,cockburn town,21.4612,-71.1419,78.60,72,0,9.01,TC,1619462888,Screaming Reels Fishing Charters & Guesthouse


In [74]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [75]:
#Converted the Hotel Name column to a list
hotel_name = hotel_df['Hotel Name'].tolist()
print(hotel_name)

['La Casa de Tounens', 'Hotel Los Inmigrantes', 'Ámbit Boulevard Hotel Boutique', 'Hotel Casa Do Capitão', 'Toscano Hotel & Café', 'Screaming Reels Fishing Charters & Guesthouse']


In [76]:
#Created marker layer
hotel_layer = gmaps.marker_layer(
    locations,
    info_box_content=hotel_info
)

fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [77]:
# Combined heat layer and marker layer
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))